In [1]:
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import torch.optim as optim
import torch
from poutyne import Model

In [2]:
import torch
torch.manual_seed(5321)
#torch.set_deterministic(True)

In [3]:
# Dimensions of input and output
dim = 4
rs = np.random.RandomState(412112)
hidden_layers = 3
hidden_units = dim

# Learnable backward net (inverse rules of the game that we need to learn)
class BackwardNet(nn.Module):
    def __init__(self):
        super(BackwardNet, self).__init__()
        self.hidden_layers = [nn.Linear(dim if i == 0 else hidden_units, hidden_units) for i in range(hidden_layers)]
        self.output_layer = nn.Linear(hidden_units, dim)

    def forward(self, x):
        for layer in self.hidden_layers:
            x = F.relu(layer(x))
        x = torch.sigmoid(self.output_layer(x))
        return x

# Fixed forward net (rules of the game)
class ForwardNet(nn.Module):
    def __init__(self):
        super(ForwardNet, self).__init__()
        self.w1 = torch.tensor(rs.choice([1, -1.1], size=(dim, dim)), dtype=torch.float, requires_grad=False)
        self.b1 = torch.tensor(rs.choice([0, -0.5], size=dim), dtype=torch.float, requires_grad=False)
        self.w2 = torch.tensor(rs.choice([1, -1.1], size=(dim, dim)), dtype=torch.float, requires_grad=False)
        self.b2 = torch.tensor(rs.choice([0, -0.5], size=dim), dtype=torch.float, requires_grad=False)

    def forward(self, x):
        x = F.relu(x @ self.w1 + self.b1)
        x = torch.sigmoid(x @ self.w2 + self.b2)
        return x

class FullNet(nn.Module):
    def __init__(self):
        super(FullNet, self).__init__()
        self.backward_net = BackwardNet()
        self.forward_net = ForwardNet()

    def forward(self, x):
        x = self.backward_net.forward(x)
        x = self.forward_net.forward(x)
        return x

In [4]:
net = FullNet()
print(net)

criterion = nn.BCELoss()
optimizer = optim.Adam(net.parameters(), lr=0.01)

model = Model(net, optimizer, criterion)

m = 10
X = torch.tensor(rs.choice([1, 0], size=(m, dim)), dtype=torch.float)


FullNet(
  (backward_net): BackwardNet(
    (output_layer): Linear(in_features=4, out_features=4, bias=True)
  )
  (forward_net): ForwardNet()
)


In [5]:
# Example of how game rules make step foward (just to make sure our rules don't output all zeros or all ones etc.)
Y = net.forward_net(X).round()
print('X:')
print(X)
print('Step forward from X:')
print(Y)

X:
tensor([[1., 0., 1., 1.],
        [1., 1., 1., 1.],
        [1., 0., 0., 0.],
        [1., 0., 1., 1.],
        [0., 1., 0., 0.],
        [0., 0., 1., 0.],
        [1., 0., 0., 1.],
        [1., 0., 1., 1.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.]])
Step forward from X:
tensor([[1., 1., 1., 1.],
        [1., 0., 1., 1.],
        [1., 0., 0., 0.],
        [1., 1., 1., 1.],
        [1., 0., 0., 0.],
        [1., 1., 0., 1.],
        [1., 0., 1., 1.],
        [1., 1., 1., 1.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.]])


In [6]:
X_hat = model.predict(X).round()

print('X:')
print(X)

print('X_hat before training:')
print(X_hat)

mae_0 = torch.mean(torch.abs(X - X_hat))
print(f'MAE before training: {mae_0}')

X:
tensor([[1., 0., 1., 1.],
        [1., 1., 1., 1.],
        [1., 0., 0., 0.],
        [1., 0., 1., 1.],
        [0., 1., 0., 0.],
        [0., 0., 1., 0.],
        [1., 0., 0., 1.],
        [1., 0., 1., 1.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.]])
X_hat before training:
[[1. 1. 1. 1.]
 [1. 1. 1. 1.]
 [1. 0. 1. 1.]
 [1. 1. 1. 1.]
 [1. 0. 1. 1.]
 [1. 0. 1. 1.]
 [1. 0. 1. 1.]
 [1. 1. 1. 1.]
 [1. 0. 1. 1.]
 [1. 0. 1. 1.]]
MAE before training: 0.44999998807907104


In [7]:
model.fit(X, X, epochs=1000)

Epoch: 1/1000 Step: 1/1 100.00% |█████████████████████████|0.13s loss: 0.691700691700
Epoch: 2/1000 Step: 1/1 100.00% |█████████████████████████|0.01s loss: 0.690110690110
Epoch: 3/1000 Step: 1/1 100.00% |█████████████████████████|0.01s loss: 0.688543688543
Epoch: 4/1000 Step: 1/1 100.00% |█████████████████████████|0.01s loss: 0.687000687000
Epoch: 5/1000 Step: 1/1 100.00% |█████████████████████████|0.00s loss: 0.685481685481
Epoch: 6/1000 Step: 1/1 100.00% |█████████████████████████|0.00s loss: 0.683987683987
Epoch: 7/1000 Step: 1/1 100.00% |█████████████████████████|0.00s loss: 0.682518682518
Epoch: 8/1000 Step: 1/1 100.00% |█████████████████████████|0.00s loss: 0.681076681076
Epoch: 9/1000 Step: 1/1 100.00% |█████████████████████████|0.00s loss: 0.679660679660
Epoch: 10/1000 Step: 1/1 100.00% |█████████████████████████|0.00s loss: 0.678271678271
Epoch: 11/1000 Step: 1/1 100.00% |█████████████████████████|0.00s loss: 0.676908676908
Epoch: 12/1000 Step: 1/1 100.00% |██████████████████

Epoch: 59/1000 Step: 1/1 100.00% |█████████████████████████|0.00s loss: 0.642796642796
Epoch: 60/1000 Step: 1/1 100.00% |█████████████████████████|0.00s loss: 0.642585642585
Epoch: 61/1000 Step: 1/1 100.00% |█████████████████████████|0.00s loss: 0.642378642378
Epoch: 62/1000 Step: 1/1 100.00% |█████████████████████████|0.00s loss: 0.642174642174
Epoch: 63/1000 Step: 1/1 100.00% |█████████████████████████|0.00s loss: 0.641972641972
Epoch: 64/1000 Step: 1/1 100.00% |█████████████████████████|0.00s loss: 0.641773641773
Epoch: 65/1000 Step: 1/1 100.00% |█████████████████████████|0.00s loss: 0.641575641575
Epoch: 66/1000 Step: 1/1 100.00% |█████████████████████████|0.00s loss: 0.641378641378
Epoch: 67/1000 Step: 1/1 100.00% |█████████████████████████|0.00s loss: 0.641182641182
Epoch: 68/1000 Step: 1/1 100.00% |█████████████████████████|0.00s loss: 0.640988640988
Epoch: 69/1000 Step: 1/1 100.00% |█████████████████████████|0.00s loss: 0.640793640793
Epoch: 70/1000 Step: 1/1 100.00% |█████████

Epoch: 116/1000 Step: 1/1 100.00% |█████████████████████████|0.00s loss: 0.631474631474
Epoch: 117/1000 Step: 1/1 100.00% |█████████████████████████|0.00s loss: 0.631273631273
Epoch: 118/1000 Step: 1/1 100.00% |█████████████████████████|0.00s loss: 0.631071631071
Epoch: 119/1000 Step: 1/1 100.00% |█████████████████████████|0.00s loss: 0.630870630870
Epoch: 120/1000 Step: 1/1 100.00% |█████████████████████████|0.00s loss: 0.630669630669
Epoch: 121/1000 Step: 1/1 100.00% |█████████████████████████|0.00s loss: 0.630467630467
Epoch: 122/1000 Step: 1/1 100.00% |█████████████████████████|0.00s loss: 0.630265630265
Epoch: 123/1000 Step: 1/1 100.00% |█████████████████████████|0.00s loss: 0.630064630064
Epoch: 124/1000 Step: 1/1 100.00% |█████████████████████████|0.00s loss: 0.629863629863
Epoch: 125/1000 Step: 1/1 100.00% |█████████████████████████|0.00s loss: 0.629661629661
Epoch: 126/1000 Step: 1/1 100.00% |█████████████████████████|0.00s loss: 0.629460629460
Epoch: 127/1000 Step: 1/1 100.00

Epoch: 173/1000 Step: 1/1 100.00% |█████████████████████████|0.00s loss: 0.620006620006
Epoch: 174/1000 Step: 1/1 100.00% |█████████████████████████|0.00s loss: 0.619806619806
Epoch: 175/1000 Step: 1/1 100.00% |█████████████████████████|0.00s loss: 0.619606619606
Epoch: 176/1000 Step: 1/1 100.00% |█████████████████████████|0.00s loss: 0.619406619406
Epoch: 177/1000 Step: 1/1 100.00% |█████████████████████████|0.00s loss: 0.619206619206
Epoch: 178/1000 Step: 1/1 100.00% |█████████████████████████|0.00s loss: 0.619007619007
Epoch: 179/1000 Step: 1/1 100.00% |█████████████████████████|0.00s loss: 0.618807618807
Epoch: 180/1000 Step: 1/1 100.00% |█████████████████████████|0.00s loss: 0.618607618607
Epoch: 181/1000 Step: 1/1 100.00% |█████████████████████████|0.00s loss: 0.618408618408
Epoch: 182/1000 Step: 1/1 100.00% |█████████████████████████|0.00s loss: 0.618209618209
Epoch: 183/1000 Step: 1/1 100.00% |█████████████████████████|0.00s loss: 0.618010618010
Epoch: 184/1000 Step: 1/1 100.00

Epoch: 230/1000 Step: 1/1 100.00% |█████████████████████████|0.00s loss: 0.608765608765
Epoch: 231/1000 Step: 1/1 100.00% |█████████████████████████|0.00s loss: 0.608571608571
Epoch: 232/1000 Step: 1/1 100.00% |█████████████████████████|0.00s loss: 0.608377608377
Epoch: 233/1000 Step: 1/1 100.00% |█████████████████████████|0.00s loss: 0.608183608183
Epoch: 234/1000 Step: 1/1 100.00% |█████████████████████████|0.00s loss: 0.607990607990
Epoch: 235/1000 Step: 1/1 100.00% |█████████████████████████|0.00s loss: 0.607796607796
Epoch: 236/1000 Step: 1/1 100.00% |█████████████████████████|0.00s loss: 0.607603607603
Epoch: 237/1000 Step: 1/1 100.00% |█████████████████████████|0.00s loss: 0.607410607410
Epoch: 238/1000 Step: 1/1 100.00% |█████████████████████████|0.00s loss: 0.607217607217
Epoch: 239/1000 Step: 1/1 100.00% |█████████████████████████|0.00s loss: 0.607024607024
Epoch: 240/1000 Step: 1/1 100.00% |█████████████████████████|0.00s loss: 0.606831606831
Epoch: 241/1000 Step: 1/1 100.00

...

Epoch: 800/1000 Step: 1/1 100.00% |█████████████████████████|0.00s loss: 0.524273524273
Epoch: 801/1000 Step: 1/1 100.00% |█████████████████████████|0.00s loss: 0.524181524181
Epoch: 802/1000 Step: 1/1 100.00% |█████████████████████████|0.00s loss: 0.524085524085
Epoch: 803/1000 Step: 1/1 100.00% |█████████████████████████|0.00s loss: 0.523986523986
Epoch: 804/1000 Step: 1/1 100.00% |█████████████████████████|0.00s loss: 0.523884523884
Epoch: 805/1000 Step: 1/1 100.00% |█████████████████████████|0.00s loss: 0.523780523780
Epoch: 806/1000 Step: 1/1 100.00% |█████████████████████████|0.00s loss: 0.523673523673
Epoch: 807/1000 Step: 1/1 100.00% |█████████████████████████|0.00s loss: 0.523563523563
Epoch: 808/1000 Step: 1/1 100.00% |█████████████████████████|0.00s loss: 0.523452523452
Epoch: 809/1000 Step: 1/1 100.00% |█████████████████████████|0.00s loss: 0.523486523486
Epoch: 810/1000 Step: 1/1 100.00% |█████████████████████████|0.00s loss: 0.523258523258
Epoch: 811/1000 Step: 1/1 100.00

Epoch: 857/1000 Step: 1/1 100.00% |█████████████████████████|0.00s loss: 0.518762518762
Epoch: 858/1000 Step: 1/1 100.00% |█████████████████████████|0.00s loss: 0.518659518659
Epoch: 859/1000 Step: 1/1 100.00% |█████████████████████████|0.00s loss: 0.518569518569
Epoch: 860/1000 Step: 1/1 100.00% |█████████████████████████|0.00s loss: 0.518480518480
Epoch: 861/1000 Step: 1/1 100.00% |█████████████████████████|0.00s loss: 0.518403518403
Epoch: 862/1000 Step: 1/1 100.00% |█████████████████████████|0.00s loss: 0.518320518320
Epoch: 863/1000 Step: 1/1 100.00% |█████████████████████████|0.00s loss: 0.518232518232
Epoch: 864/1000 Step: 1/1 100.00% |█████████████████████████|0.00s loss: 0.518140518140
Epoch: 865/1000 Step: 1/1 100.00% |█████████████████████████|0.00s loss: 0.518044518044
Epoch: 866/1000 Step: 1/1 100.00% |█████████████████████████|0.00s loss: 0.517945517945
Epoch: 867/1000 Step: 1/1 100.00% |█████████████████████████|0.00s loss: 0.517843517843
Epoch: 868/1000 Step: 1/1 100.00

Epoch: 914/1000 Step: 1/1 100.00% |█████████████████████████|0.00s loss: 0.513665513665
Epoch: 915/1000 Step: 1/1 100.00% |█████████████████████████|0.00s loss: 0.513592513592
Epoch: 916/1000 Step: 1/1 100.00% |█████████████████████████|0.00s loss: 0.513513513513
Epoch: 917/1000 Step: 1/1 100.00% |█████████████████████████|0.00s loss: 0.513428513428
Epoch: 918/1000 Step: 1/1 100.00% |█████████████████████████|0.00s loss: 0.513339513339
Epoch: 919/1000 Step: 1/1 100.00% |█████████████████████████|0.00s loss: 0.513246513246
Epoch: 920/1000 Step: 1/1 100.00% |█████████████████████████|0.00s loss: 0.513149513149
Epoch: 921/1000 Step: 1/1 100.00% |█████████████████████████|0.00s loss: 0.513119513119
Epoch: 922/1000 Step: 1/1 100.00% |█████████████████████████|0.00s loss: 0.512984512984
Epoch: 923/1000 Step: 1/1 100.00% |█████████████████████████|0.00s loss: 0.512913512913
Epoch: 924/1000 Step: 1/1 100.00% |█████████████████████████|0.00s loss: 0.512836512836
Epoch: 925/1000 Step: 1/1 100.00

Epoch: 971/1000 Step: 1/1 100.00% |█████████████████████████|0.00s loss: 0.509147509147
Epoch: 972/1000 Step: 1/1 100.00% |█████████████████████████|0.00s loss: 0.509051509051
Epoch: 973/1000 Step: 1/1 100.00% |█████████████████████████|0.00s loss: 0.508983508983
Epoch: 974/1000 Step: 1/1 100.00% |█████████████████████████|0.00s loss: 0.509021509021
Epoch: 975/1000 Step: 1/1 100.00% |█████████████████████████|0.00s loss: 0.508823508823
Epoch: 976/1000 Step: 1/1 100.00% |█████████████████████████|0.00s loss: 0.508777508777
Epoch: 977/1000 Step: 1/1 100.00% |█████████████████████████|0.00s loss: 0.508719508719
Epoch: 978/1000 Step: 1/1 100.00% |█████████████████████████|0.00s loss: 0.508650508650
Epoch: 979/1000 Step: 1/1 100.00% |█████████████████████████|0.00s loss: 0.508571508571
Epoch: 980/1000 Step: 1/1 100.00% |█████████████████████████|0.00s loss: 0.508482508482
Epoch: 981/1000 Step: 1/1 100.00% |█████████████████████████|0.00s loss: 0.508410508410
Epoch: 982/1000 Step: 1/1 100.00

[{'epoch': 1, 'loss': 0.691700279712677, 'time': 0.1321707},
 {'epoch': 2, 'loss': 0.6901101469993591, 'time': 0.008543300000000004},
 {'epoch': 3, 'loss': 0.6885432004928589, 'time': 0.005998699999999996},
 {'epoch': 4, 'loss': 0.687000036239624, 'time': 0.006149099999999991},
 {'epoch': 5, 'loss': 0.6854811310768127, 'time': 0.004104099999999999},
 {'epoch': 6, 'loss': 0.6839872598648071, 'time': 0.004164399999999985},
 {'epoch': 7, 'loss': 0.682518482208252, 'time': 0.0030739000000000183},
 {'epoch': 8, 'loss': 0.6810759902000427, 'time': 0.0030941000000000163},
 {'epoch': 9, 'loss': 0.6796598434448242, 'time': 0.003364800000000001},
 {'epoch': 10, 'loss': 0.6782705187797546, 'time': 0.003960899999999989},
 {'epoch': 11, 'loss': 0.6769083738327026, 'time': 0.002792799999999984},
 {'epoch': 12, 'loss': 0.6755732297897339, 'time': 0.003710700000000011},
 {'epoch': 13, 'loss': 0.6742659211158752, 'time': 0.0030347000000000013},
 {'epoch': 14, 'loss': 0.6729861497879028, 'time': 0.00362

In [8]:
model.predict(X)

array([[0.8070521 , 0.17163771, 0.71727157, 0.71727157],
       [0.7803937 , 0.19864918, 0.6830799 , 0.6830799 ],
       [0.616763  , 0.37205413, 0.49395812, 0.49395812],
       [0.8070521 , 0.17163771, 0.71727157, 0.71727157],
       [0.53055763, 0.46639538, 0.40670177, 0.40670177],
       [0.5482085 , 0.44700527, 0.4239533 , 0.4239533 ],
       [0.69649   , 0.28624275, 0.5819148 , 0.5819148 ],
       [0.8070521 , 0.17163771, 0.71727157, 0.71727157],
       [0.50005037, 0.49994457, 0.37758803, 0.37758803],
       [0.50005037, 0.49994457, 0.37758803, 0.37758803]], dtype=float32)

In [9]:
X_hat = model.predict(X).round()
mae_1 = torch.mean(torch.abs(X - X_hat))
print(f'MAE after training: {mae_1} (improvement: {mae_1 - mae_0})')


print('X:')
print(X)

print('X_hat after training:')
print(X_hat)


MAE after training: 0.20000000298023224 (improvement: -0.2499999850988388)
X:
tensor([[1., 0., 1., 1.],
        [1., 1., 1., 1.],
        [1., 0., 0., 0.],
        [1., 0., 1., 1.],
        [0., 1., 0., 0.],
        [0., 0., 1., 0.],
        [1., 0., 0., 1.],
        [1., 0., 1., 1.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.]])
X_hat after training:
[[1. 0. 1. 1.]
 [1. 0. 1. 1.]
 [1. 0. 0. 0.]
 [1. 0. 1. 1.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 1. 1.]
 [1. 0. 1. 1.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]]
